In [1]:
import numpy as np
import pandas as pd
import math

low_memory = False

# Read csv into pandas dataframe

df = pd.read_csv('train.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
np.sort(df['VisitorTeamAbbr'].unique())

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [3]:
np.sort(df['HomeTeamAbbr'].unique())

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [4]:
np.sort(df['PossessionTeam'].unique())

array(['ARZ', 'ATL', 'BLT', 'BUF', 'CAR', 'CHI', 'CIN', 'CLV', 'DAL',
       'DEN', 'DET', 'GB', 'HST', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [5]:
df['FieldPosition'].unique()

array(['NE', 'KC', nan, 'BUF', 'NYJ', 'ATL', 'CHI', 'CIN', 'BLT', 'CLV',
       'PIT', 'ARZ', 'DET', 'JAX', 'HST', 'TEN', 'OAK', 'WAS', 'PHI',
       'LA', 'IND', 'GB', 'SEA', 'CAR', 'SF', 'DAL', 'NYG', 'NO', 'MIN',
       'DEN', 'LAC', 'TB', 'MIA'], dtype=object)

In [6]:
df = df.replace({'PossessionTeam': {'BLT': 'BAL', 'CLV': 'CLE', 'HST': 'HOU', 'ARZ': 'ARI'}})
df = df.replace({'FieldPosition': {'BLT': 'BAL', 'CLV': 'CLE', 'HST': 'HOU', 'ARZ': 'ARI'}})

In [7]:
pd.set_option('display.max_columns', None)

df['FieldPosition'].unique()

array(['NE', 'KC', nan, 'BUF', 'NYJ', 'ATL', 'CHI', 'CIN', 'BAL', 'CLE',
       'PIT', 'ARI', 'DET', 'JAX', 'HOU', 'TEN', 'OAK', 'WAS', 'PHI',
       'LA', 'IND', 'GB', 'SEA', 'CAR', 'SF', 'DAL', 'NYG', 'NO', 'MIN',
       'DEN', 'LAC', 'TB', 'MIA'], dtype=object)

In [8]:
# Kaggle data modified with adjusted Player Names
kaggle_df = df.copy()
kaggle_df['DisplayName'] = kaggle_df['DisplayName'].str.lower().str.replace(" ", ""
                                                                ).str.replace('[^\w\s]',''
                                                                             )
# Madden Data loaded
nfl_17 = pd.read_csv("nfl_rating_17 v2.csv")
nfl_18 = pd.read_csv("nfl_rating_18 v2.csv")
nfl_19 = pd.read_csv("nfl_rating_19 v2.csv")

In [9]:
# Player name isolation based on season
kaggle_17 = kaggle_df[kaggle_df['Season']==2017]
name_17 = kaggle_17['DisplayName'].drop_duplicates().to_list()

kaggle_18 = kaggle_df[kaggle_df['Season']==2018]
name_18 = kaggle_18['DisplayName'].drop_duplicates().to_list()

kaggle_19 = kaggle_df[kaggle_df['Season']==2019]
name_19 = kaggle_19['DisplayName'].drop_duplicates().to_list()

In [10]:
# Players that will obtain Madden data
yes_show_17 = []
for i in name_17:
    if i in nfl_17['name'].tolist():
        yes_show_17.append(i)

yes_show_18 = []
for i in name_18:
    if i in nfl_18['name'].tolist():
        yes_show_18.append(i)

yes_show_19 = []
for i in name_19:
    if i in nfl_19['name'].tolist():
        yes_show_19.append(i)

In [11]:
# Madden columns currently being used for merge
column_order = ['name','awareness',
       'strength', 'ball carrier vision',
       'pursuit', 'acceleration', 'spin move',
       'finesse moves', 'run block', 'tackle',
       'zone coverage', 'weight', 'truck', 'juke move',
       'play recognition', 'speed', 'run block power', 'jumping', 'toughness',
       'man coverage', 'stiff arm', 'power moves',
       'release', 'hit power',
       'pass block power', 'impact blocking', 'stamina', 'carrying','pass block', 'press',
       'block shedding', 'run block finesse', 'agility', 'elusiveness',
       'pass block finesse', 'route running']

In [12]:
# Madden datasets adjusted to show names that appear in Kaggle dataset
# Kaggle dataset has 'DisplayName' column renamed to 'name' for merger purposes

nfl_17_mod = nfl_17[nfl_17['name'].isin(yes_show_17)].drop(columns=['Unnamed: 0'])
kaggle_17 = kaggle_17.rename(columns={'DisplayName':'name'})
nfl_17_mod = nfl_17_mod[column_order]

nfl_18_mod = nfl_18[nfl_18['name'].isin(yes_show_18)].drop(columns=['Unnamed: 0'])
kaggle_18 = kaggle_18.rename(columns={'DisplayName':'name'})
nfl_18_mod = nfl_18_mod[column_order]

nfl_19_mod = nfl_19[nfl_19['name'].isin(yes_show_19)].drop(columns=['Unnamed: 0'])
kaggle_19 = kaggle_19.rename(columns={'DisplayName':'name'})
nfl_19_mod = nfl_19_mod[column_order]

In [13]:
# Merge between Madden and Kaggle 
nfl_17_res = kaggle_17.merge(nfl_17_mod, on='name', how='left')
nfl_18_res = kaggle_18.merge(nfl_18_mod, on='name', how='left')
nfl_19_res = kaggle_19.merge(nfl_19_mod, on='name', how='left')

kaggle_modified = pd.concat([nfl_17_res, nfl_18_res, nfl_19_res], ignore_index=True)

In [14]:
kaggle_modified.to_csv("Kaggle_Madden.csv")

In [15]:
# Resulting merge dataframe to CSV file
# nfl_17_res.to_csv("kaggle_17_Complete.csv")
# nfl_18_res.to_csv("kaggle_18_Complete.csv")
# nfl_19_res.to_csv("kaggle_19_Complete.csv")